In [1]:
import os
import asyncio
from pinecone import Pinecone
from dotenv import load_dotenv


load_dotenv()

os.environ["MISTRAL_API_KEY"] = os.getenv("MISTRAL_AI_API")

from langchain_mistralai import ChatMistralAI

model = ChatMistralAI(model="mistral-large-latest")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
from datasets import load_dataset
from langchain_mistralai import MistralAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

# Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [16]:
import time
from langchain_pinecone import PineconeVectorStore
from pinecone import ServerlessSpec

index_name = "l6v2"  # change if desired

vector_store= PineconeVectorStore(index_name, embedding=embedding_model.aembed_query)

retriever = vector_store.as_retriever()

In [17]:

from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.messages import SystemMessage, trim_messages
from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a professional Gym/Health Trainer . Answer all questions to the best of your ability in {language} using the following context: {context}, Don't Answer anything unrelated to fitness/health",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [10]:
from typing import List
from langchain_core.messages import HumanMessage

index = pc.Index(index_name)

# Assuming `retriever` is set up as shown in previous answers with Pinecone
async def get_context_from_pinecone(query: str, top_k: int = 5) -> str:
    """Retrieve relevant context from Pinecone based on the query."""
    # Generate query embedding
    query_embedding = embedding_model.embed_query(query)

    # Query Pinecone for relevant documents
    response = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True,
        namespace="fitness-chatbot"
    )

    # Extract context from retrieved documents
    context = " ".join(
        [match['metadata']['output'] for match in response['matches']]
    ) if response['matches'] else "No relevant context found."

    return context

# Define the graph
workflow = StateGraph(state_schema=State)

async def call_model(state: State):
    # Define the chain with your prompt and language model
    chain = prompt | model

    # Extract the user's latest message content
    query = state["messages"][-1].content

    # Retrieve context from the fitness dataset using RAG
    try:
        context = await get_context_from_pinecone(query)
    except Exception as e:
        print(f"Error retrieving documents: {e}")
        context = "An error occurred while retrieving relevant context."

    # Trim and prepare messages with context for the model
    trimmed_messages = trimmer.invoke(state["messages"])

    # Generate response using context from retrieved docs
    try:
        response = await chain.ainvoke(
            {
                "messages": trimmed_messages,  # User's message history
                "language": state["language"],  # User's language preference
                "context": context              # Context from fitness dataset
            }
        )
    except Exception as e:
        print(f"Error generating response: {e}")
        return {"messages": [HumanMessage(content="An error occurred while generating the response.")]}

    return {"messages": [response]}

# Workflow Details
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()

app = workflow.compile(checkpointer=memory)


In [11]:
config = {"configurable": {"thread_id": "abc1"}}
query = "Hi! I'm Vkay."
language = "English"

input_messages = [HumanMessage(query)]

async for chunk, metadata in app.astream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Hello| V|k|ay|!| How| can| I| assist| you| today| with| your| fitness| or| health| goals|?||

In [12]:
config = {"configurable": {"thread_id": "abc1"}}
query = "Hi! I'm on my way to the gym, I will be hitting chest today. Give me a workout schedule."
language = "English"

input_messages = [HumanMessage(query)]

async for chunk, metadata in app.astream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="")

Great to hear that you're headed to the gym! Here's a simple and effective chest workout for you:

**Warm-up:**

* 5-10 minutes of light cardio (treadmill, stationary bike, or rowing machine)
* Dynamic stretches (arm circles, cat-cow stretch, and shoulder rolls)

**Workout:**

1. **Barbell Bench Press:**
	* 4 sets of 8-12 reps
	* Rest for 60-90 seconds between sets
2. **Incline Dumbbell Press:**
	* 3 sets of 10-12 reps
	* Rest for 60 seconds between sets
3. **Cable Flyes:**
	* 3 sets of 12-15 reps
	* Rest for 60 seconds between sets
4. **Push-ups:**
	* 3 sets to failure
	* Rest for 60 seconds between sets
5. **Dumbbell Pullovers:**
	* 2 sets of 12-15 reps
	* Rest for 60 seconds between sets

**Cool-down:**

* Static stretches for the chest, shoulders, and triceps

**Tips:**

* Start with a weight that challenges you but allows you to complete the suggested repetitions with good form.
* Increase the weight or resistance as you get stronger.
* Make sure to engage your core and maintain p

In [21]:
config = {"configurable": {"thread_id": "abc1"}}
query = "ugygvghcuyuibujbhuybnyunb"
language = "English"

input_messages = [HumanMessage(query)]

async for chunk, metadata in app.astream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="")

Of course, I'm ready to assist with your fitness and health questions. Please provide a clear question or topic related to exercise, nutrition, or wellness, and I'll do my best to provide a helpful and informative response in the following format:

Output: [Answer to your question]

Let's start whenever you're ready.